## Data Collection Notebook

**Objectives:**
* Load UK housing dataset from Kaggle
* Examine data structure and quality
* Save clean data for analysis

**Inputs**:
*uk_housing_data.csv from inputs/datasets/raw

**Outputs:**
* Clean dataset in inputs/dataset/collection/
* Data summary report

In [1]:
# Setup Kaggle API credentials
import os
import json

kaggle_credentials = {
    "username": "ahmedgodah",
    "key": "68faed8df5add337f9b9169e619b560b"
}

kaggle_dir = os.path.expanduser("~/.kaggle")
os.makedirs(kaggle_dir, exist_ok=True)

kaggle_json_path = os.path.join(kaggle_dir, "kaggle.json")
with open(kaggle_json_path, "w") as f:
    json.dump(kaggle_credentials, f)
os.chmod(kaggle_json_path, 0o600)
print("Kaggle credentials configured")

Kaggle credentials configured


In [2]:
!pip install kaggle
!kaggle datasets download -d hm-land-registry/uk-housing-prices-paid -p ../inputs/datasets/raw/
print("Dataset downloaded")


Dataset URL: https://www.kaggle.com/datasets/hm-land-registry/uk-housing-prices-paid
License(s): other
uk-housing-prices-paid.zip: Skipping, found more recently modified local copy (use --force to force download)
Dataset downloaded


In [3]:
import pandas as pd
import numpy as np

In [4]:
import zipfile
from pathlib import Path

zip_files = list(Path("../inputs/datasets/raw/").glob("*.zip"))
if zip_files:
    with zipfile.ZipFile(zip_files[0], 'r') as zip_ref:
        zip_ref.extractall("../inputs/datasets/raw/")

    csv_files = list(Path("../inputs/datasets/raw/").glob("*.csv"))
    df = pd.read_csv(csv_files[0], low_memory=False)
    print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")

Dataset loaded: 22489348 rows, 11 columns


In [5]:
print("Column names:")
print(df.columns.tolist())
print("\nFirst 5 rows")
df.head()

Column names:
['Transaction unique identifier', 'Price', 'Date of Transfer', 'Property Type', 'Old/New', 'Duration', 'Town/City', 'District', 'County', 'PPDCategory Type', 'Record Status - monthly file only']

First 5 rows


,Transaction unique identifier,Price,Date of Transfer,Property Type,Old/New,Duration,Town/City,District,County,PPDCategory Type,Record Status - monthly file only
0,{81B82214-7FBC-4129-9F6B-4956B4A663AD},25000,1995-08-18 00:00,T,N,F,OLDHAM,OLDHAM,GREATER MANCHESTER,A,A
1,{8046EC72-1466-42D6-A753-4956BF7CD8A2},42500,1995-08-09 00:00,S,N,F,GRAYS,THURROCK,THURROCK,A,A
2,{278D581A-5BF3-4FCE-AF62-4956D87691E6},45000,1995-06-30 00:00,T,N,F,HIGHBRIDGE,SEDGEMOOR,SOMERSET,A,A
3,{1D861C06-A416-4865-973C-4956DB12CD12},43150,1995-11-24 00:00,T,N,F,BEDFORD,NORTH BEDFORDSHIRE,BEDFORDSHIRE,A,A
4,{DD8645FD-A815-43A6-A7BA-4956E58F1874},18899,1995-06-23 00:00,S,N,F,WAKEFIELD,LEEDS,WEST YORKSHIRE,A,A


In [6]:
# Basic data information
print("Dataset Info:")
print(f"Shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

print("\nData Types:")
print(df.dtypes)

Dataset Info:
Shape: (22489348, 11)
Memory usage: 12618.3 MB

Data Types:
Transaction unique identifier        object
Price                                 int64
Date of Transfer                     object
Property Type                        object
Old/New                              object
Duration                             object
Town/City                            object
District                             object
County                               object
PPDCategory Type                     object
Record Status - monthly file only    object
dtype: object


In [7]:
# Check for missing values
print("Missing Values:")
missing_data = df.isnull().sum()
print(missing_data)
print(f"\nTotal missing values: {missing_data.sum()}")

Missing Values:
Transaction unique identifier        0
Price                                0
Date of Transfer                     0
Property Type                        0
Old/New                              0
Duration                             0
Town/City                            0
District                             0
County                               0
PPDCategory Type                     0
Record Status - monthly file only    0
dtype: int64

Total missing values: 0


In [8]:
# Analyze price column
print("Price Statistics:")
print(df['Price'].describe)

print(f"\nPrice range: £{df['Price'].min():,} to £{df['Price'].max():,}")
print(f"Average price: £{df['Price'].mean():,.0f}")
print(f"Median price: £{df['Price'].median():,.0f}")


Price Statistics:
<bound method NDFrame.describe of 0            25000
1            42500
2            45000
3            43150
4            18899
             ...  
22489343    175000
22489344    586945
22489345    274000
22489346     36000
22489347    145000
Name: Price, Length: 22489348, dtype: int64>

Price range: £1 to £98,900,000
Average price: £178,244
Median price: £130,000


In [9]:
# Property type distribution
print("Property Types:")
print(df['Property Type'].value_counts())


Property Types:
Property Type
T    6918811
S    6216218
D    5170327
F    4083424
O     100568
Name: count, dtype: int64


In [10]:
# Save to collection folder for analysis
output_path = "../inputs/datasets/collection/uk_housing_clean.csv"
df.to_csv(output_path, index=False)
print(f"Clean dataset saved to: {output_path}")

Clean dataset saved to: ../inputs/datasets/collection/uk_housing_clean.csv


In [11]:
# create a smaller database for fast loading
df_small = df.sample(n=20000, random_state=42)
df_small.to_csv("../inputs/datasets/collection/uk_housing_small.csv", index=False)
print(f"Small dataset created with {len(df_small):,} properties")

Small dataset created with 20,000 properties
